In [1]:
import numpy as np
import datetime
import netCDF4
import gsw
#-------------------------------------------------------------------------|
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import matplotlib.colors as colors
import cmocean
import cmocean.cm as cmo
#-------------------------------------------------------------------------|
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from scipy.io import loadmat
import xarray as xr
import math
import h5py
import pandas as pd
import re
from pathlib import Path
import os
import numpy.ma as ma
import subprocess
import tqdm as tqdm
###############################
from pyresample import kd_tree, geometry, utils
from pyresample.geometry import GridDefinition
import pyproj as pyproj
from  pyproj import transform
from pyproj import Proj
# may make plotting faster
pyproj.set_use_global_context()
###############################
import sys
sys.path.append(r'/Users/houndegno/Documents/JupyterNoteBook/Prog/odi')
import odi
#-------------------------------------------------------------------------|
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Function of interpolatio  into 70 levels

In [2]:
def ts_to_70levels(T,S,new_z):
    # to be retuned
    Temp       = T.interp(depth=new_z,kwargs={"fill_value": "extrapolate"})
    Sal        = S.interp(depth=new_z,kwargs={"fill_value": "extrapolate"})
    return Temp,Sal

# Function to ferifying the interpolation after interpolation "action"

In [3]:
def verif_plot_ts_to_70levels(T,S,z,new_z,lon,lat,time,rep_save,n,sFig):
    # to be retuned after interpolation
    T2,S2 = ts_to_70levels(T,S,new_z)
    #---------------------------------------------------------------------#
    fig = plt.figure(figsize=[10,6], num=2);plt.clf();
    #---------------------------------------------------------------------#
    a = plt.subplot(1,2,1)
    a.plot(S,z,label='init S')
    a.plot(S2,new_z,label='interp S')
    a.invert_yaxis()
    plt.legend()
    a.set_xlabel('Absolute Salinity')
    a.set_ylabel('Depth')
    plt.title(str(np.array(time))[slice(0,19,1)])
    #---------------------------------------------------------------------#
    b = plt.subplot(1,2,2)
    b.plot(T,z,label='init T')
    b.plot(T2,new_z,label='interp T')
    b.invert_yaxis()
    plt.legend()
    b.set_xlabel('Conservative Temperature')
    b.set_ylabel('Depth')
    plt.title(str(np.array(time))[slice(0,19,1)])
    del(a,b)
    #---------------------------------------------------------------------#
    if sFig == True:
        # setting lon and lat str() format
        a = np.array(lon).round(2)
        b = np.array(lat).round(2)
        if a>0:
            lonstr = str(a)+'E'
        else:
            lonstr = str(np.abs(a))+'W'
        if b>0:
            latstr = str(b)+'N'
        else:
            latstr = str(np.abs(a))+'S'
        # date str() format
        ts_date = str(np.array(time))[slice(0,10,1)]
        # filename
        ifolder = "_"+str(n).zfill(3)
        fig_ts_name = "TS_Lon_"+lonstr+"_Lat_"+latstr+"_date_"+ts_date+ifolder+".png"
        plt.savefig(rep_save+fig_ts_name)
        del(a,b)
    else:
        print(" ")

# Function to converting T and S  into .bin file

In [4]:
def ts_to_bin(T,S,lon,lat,time,rep_save):
    # setting lon and lat str() format
    a = np.array(lon).round(2)
    b = np.array(lat).round(2)
    if a>0:
        lonstr = str(a)+'E'
    else:
        lonstr = str(np.abs(a))+'W'
    if b>0:
        latstr = str(b)+'N'
    else:
        latstr = str(np.abs(a))+'S'
    # date str() format
    ts_date = str(np.array(time))[slice(0,10,1)]
    # filename
    tRefname = "tRef_Lon_"+lonstr+"_Lat_"+latstr+"_date_"+ts_date+".bin"
    sRefname = "sRef_Lon_"+lonstr+"_Lat_"+latstr+"_date_"+ts_date+".bin"
    # saving to 'rep_save'
    ## for temperature
    np.array(T).astype('>f4').tofile(rep_save+tRefname)
    ## for salinity
    np.array(S).astype('>f4').tofile(rep_save+sRefname)

# Modifiying "data" file to insert proper "tRef" and "sRef"

In [5]:
def data_modifying(filename,lon,lat,time):
    # lines to be changed
    from_pattern0 = " nTimeSteps= 3192,"
    from_pattern1 = " hydrogThetaFile = 'TEmpRef.bin',"
    from_pattern2 = " hydrogSaltFile = 'SAlRef.bin',"
    # setting lon and lat str() format ----------------
    a = np.array(lon).round(2)
    b = np.array(lat).round(2)
    if a>0:
        lonstr = str(a)+'E'
    else:
        lonstr = str(np.abs(a))+'W'
    if b>0:
        latstr = str(b)+'N'
    else:
        latstr = str(np.abs(a))+'S'
    # date str() format
    ts_date = str(np.array(time))[slice(0,10,1)]
    #--- For nTimeSteps  -----------------------------
    ## computing the nTimeSteps for the selected TS profile
    nTimeSteps = (datetime.datetime(2023,1,
                                    31)-datetime.datetime(int(ts_date.split('-')[0]),
                                                          int(ts_date.split('-')[1]),
                                                          int(ts_date.split('-')[2]))).days*24
    to_pattern0 = " nTimeSteps= "+str(nTimeSteps)+","
    #--- For tRef ------------------------------------
    tRefname = "tRef_Lon_"+lonstr+"_Lat_"+latstr+"_date_"+ts_date+".bin"
    to_pattern1 = " hydrogThetaFile = '"+tRefname+"',"
    #--- For sRef ------------------------------------
    sRefname = "sRef_Lon_"+lonstr+"_Lat_"+latstr+"_date_"+ts_date+".bin"
    to_pattern2 = " hydrogSaltFile = '"+sRefname+"',"
    file_init = open(filename,'r+')
    text = file_init.read()
    ## changing "nTimeSteps"
    splitted_text0 = re.split(from_pattern0,text)
    modified_text0 = to_pattern0.join(splitted_text0)    
    ## changing "tReft" files ID
    splitted_text1 = re.split(from_pattern1,modified_text0)
    modified_text1 = to_pattern1.join(splitted_text1)
    ## changing "sReft" files ID
    splitted_text2 = re.split(from_pattern2,modified_text1)
    modified_text2 = to_pattern2.join(splitted_text2)
    with open(filename,'w') as file_new:
        file_new.write(modified_text2)

# Modifiying "startDate_1" in "data.cal" file

In [6]:
def data_cal_modifying(filename,lon,lat,time):
    # line to be chanded
    from_pattern = " startDate_1=20220917,"
    ts_date = str(np.array(time))[slice(0,10,1)].split('-')
    to_pattern = " startDate_1="+ts_date[0]+ts_date[1]+ts_date[2]+","
    file_init = open(filename,'r+')
    text = file_init.read()
    splitted_text = re.split(from_pattern,text)
    modified_text = to_pattern.join(splitted_text)
    with open(filename,'w') as file_new:
        file_new.write(modified_text)

# Modifiying "data.exf" file

In [7]:
def data_exf_modifying(filename,lon,lat):
    from_pattern1 = " aqhfile           ='qv2m_1x1_four_months.bin',"
    from_pattern2 = " precipfile        ='precipcorr_1x1_four_months.bin',"
    from_pattern3 = " atempfile         ='atemp_1x1_four_months.bin',"
    from_pattern4 = " lwdownfile        ='lwgab_1x1_four_months.bin',"
    from_pattern5 = " swdownfile        ='swgdn_1x1_four_months.bin',"
    from_pattern6 = " uwindfile         ='u_10m_1x1_four_months.bin',"
    from_pattern7 = " vwindfile         ='v_10m_1x1_four_months.bin',"
    # setting lon and lat str() format -----------------------------------------------
    a = np.array(lon).round(2)
    b = np.array(lat).round(2)
    if a>0:
        lonstr = str(a)+'E'
    else:
        lonstr = str(np.abs(a))+'W'
    if b>0:
        latstr = str(b)+'N'
    else:
        latstr = str(np.abs(a))+'S'
    # Surfix of file name
    Surfix_name = "_1x1_four_months_Lon_"+lonstr+"_Lat_"+latstr+".bin',"
    #----------------------------------------------------------------------------------
    to_pattern1 = " aqhfile           ='qv2m"+Surfix_name
    to_pattern2 = " precipfile        ='precipcorr"+Surfix_name
    to_pattern3 = " atempfile         ='atemp"+Surfix_name
    to_pattern4 = " lwdownfile        ='lwgab"+Surfix_name
    to_pattern5 = " swdownfile        ='swgdn"+Surfix_name
    to_pattern6 = " uwindfile         ='u_10m"+Surfix_name
    to_pattern7 = " vwindfile         ='v_10m"+Surfix_name
    #----------------------------------------------------------------------------------
    file_init = open(filename,'r+')
    text = file_init.read()
    splitted_text = re.split(from_pattern1,text)
    modified_text1 = to_pattern1.join(splitted_text)
    splitted_text2 = re.split(from_pattern2,modified_text1)
    modified_text2 = to_pattern2.join(splitted_text2)
    splitted_text3 = re.split(from_pattern3,modified_text2)
    modified_text3 = to_pattern3.join(splitted_text3)
    splitted_text4 = re.split(from_pattern4,modified_text3)
    modified_text4 = to_pattern4.join(splitted_text4)
    splitted_text5 = re.split(from_pattern5,modified_text4)
    modified_text5 = to_pattern5.join(splitted_text5)
    splitted_text6 = re.split(from_pattern6,modified_text5)
    modified_text6 = to_pattern6.join(splitted_text6)
    splitted_text7 = re.split(from_pattern7,modified_text6)
    modified_text7 = to_pattern7.join(splitted_text7)
    with open(filename,'w') as file_new:
            file_new.write(modified_text7)

# Setting the new depth level: 70 vertical levels

In [8]:
new_z      = xr.DataArray(np.linspace(0.5, 69.5, 70), dims='depth')
new_z.name = 'depth'
print(new_z)
len(new_z)

<xarray.DataArray 'depth' (depth: 70)>
array([ 0.5,  1.5,  2.5,  3.5,  4.5,  5.5,  6.5,  7.5,  8.5,  9.5, 10.5,
       11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5, 20.5, 21.5,
       22.5, 23.5, 24.5, 25.5, 26.5, 27.5, 28.5, 29.5, 30.5, 31.5, 32.5,
       33.5, 34.5, 35.5, 36.5, 37.5, 38.5, 39.5, 40.5, 41.5, 42.5, 43.5,
       44.5, 45.5, 46.5, 47.5, 48.5, 49.5, 50.5, 51.5, 52.5, 53.5, 54.5,
       55.5, 56.5, 57.5, 58.5, 59.5, 60.5, 61.5, 62.5, 63.5, 64.5, 65.5,
       66.5, 67.5, 68.5, 69.5])
Dimensions without coordinates: depth


70

# Chosing SASSIE CTD data as initial data for the simulation

In [9]:
#-------------------------------------------------------------------------------------------------------|
## SASSIE final data collected files directory ---------------------------------------------------------|
Final_SASSIE    = '/Users/houndegno/Documents/MATLAB/Data/SASSIE_DATA/Final_SASSIE_update/'#------------|
## Data files directory --------------------------------------------------------------------------------|
CastAway_CTD    = 'CastAway_CTD/'#----------------------------------------------------------------------|
#-------------------------------------------------------------------------------------------------------|

In [10]:
## Cast away CTD data list
CastAway_CTD_content = os.listdir(Final_SASSIE+CastAway_CTD)
CastAway_CTD_content

['SASSIE_Fall_2022_SHIPBOARD_Castaway_CTD.nc']

In [11]:
CTD_cast = xr.open_dataset(Final_SASSIE+CastAway_CTD+CastAway_CTD_content[0])
CTD_cast

<xarray.Dataset>
Dimensions:      (time: 250, depth: 311)
Coordinates:
  * time         (time) datetime64[ns] 2022-09-09T15:22:59.999998208 ... 2022...
  * depth        (depth) float64 1.0 1.296 1.591 1.887 ... 92.06 92.35 92.65
    longitude    (time) float64 ...
    latitude     (time) float64 ...
Data variables:
    pressure     (depth, time) float64 ...
    temperature  (depth, time) float64 ...
    salinity     (depth, time) float64 ...
    density      (depth, time) float64 ...
Attributes: (12/54)
    DOI:                           10.5067/SASSIE-RVCTD2
    title:                         SASSIE Arctic Field Campaign Castaway Data...
    summary:                       Castaway data collected from R/V Woldstad ...
    keywords:                      EARTH SCIENCE > OCEANS > OCEAN TEMPERATURE...
    keywords_vocabulary:           NASA Global Change Master Directory (GCMD)...
    conventions:                   CF-1.8, ACDD-1.4
    ...                            ...
    geospatial_lon_max:            -145.3295382
    geospatial_lon_units:          degrees_east
    time_coverage_start:           2022-09-09T15:23:00Z
    time_coverage_end:             2022-09-19T00:27:00Z
    uuid:                          ac6f60ba-010d-472a-8fab-57b0f8483d87
    time_coverage_duration:        P09DT09H04M00S

# Extraction of profiles

In [12]:
# in situ temperature [˚C]
temp = CTD_cast.temperature
# in situ pratical salinity [PSU]
sal = CTD_cast.salinity
# longitude & latitude 
lon = CTD_cast.longitude
lat = CTD_cast.latitude
# depth/pressure
pres = CTD_cast.pressure
# time related to each profile
Time = CTD_cast.time
# conversion of pratical salinity to absolute salinity [SA,g/kg]
SA = gsw.SA_from_SP(sal,pres,lon,lat)
# conversion of temp to potential temperature [˚C]
potemp = gsw.pt_from_t(SA,temp,pres,p_ref = 0)

# Saving repository

In [13]:
Global_RUN = "/Users/houndegno/Documents/MATLAB/Data/SASSIE_DATA/MITgcm_experiments/1D_OJH/1D_ocean_ice_column/Global_RUN2/"
Global_RUN_Fig = Global_RUN+"Fig/"

# Making change to data, data.cal and data.exf for each run forlder

In [14]:
for ij in tqdm.tqdm(np.arange(lon.size)):
    ifolder = "_"+str(ij+1).zfill(3)
    ts_date = str(np.array(Time[ij]))[slice(0,10,1)]
    #Global_RUN_sel = Global_RUN+"run_"+ts_date+ifolder+"/"
    Global_RUN_sel = Global_RUN+"run"+ifolder+"/"
    T2, S2 = ts_to_70levels(potemp[:,ij],SA[:,ij],new_z)
    ts_to_bin(T2,S2,lon[ij],lat[ij],Time[ij],Global_RUN_sel)
    # modifying "data" file
    data_modifying(Global_RUN_sel+"data",lon[ij],lat[ij],Time[ij])
    # modifying "data.cal" file
    data_cal_modifying(Global_RUN_sel+"data.cal",lon[ij],lat[ij],Time[ij])
    # modifying "data.exf" file
    data_exf_modifying(Global_RUN_sel+"data.exf",lon[ij],lat[ij])
    # Figures after interpolation with the selected profil
    #verif_plot_ts_to_70levels(potemp[:,ij],SA[:,ij],pres[:,ij],new_z,lon[ij],
    #                          lat[ij],Time[ij],Global_RUN_Fig,n=ij+1,sFig=True)

100%|████████████████████████████████████████████████████| 250/250 [00:07<00:00, 33.19it/s]


# Seting a new batymetry file for each simlation folder

In [15]:
for iRunFolder in tqdm.tqdm(np.arange(250)):
    ifolder = "_"+str(iRunFolder+1).zfill(3)
    lastgood = np.where(~np.isnan(SA[:,iRunFolder]))[0][-1]
    baty = np.array(pres[lastgood,iRunFolder]).round(2)
    if baty < np.max(new_z):
        ibat = np.where(new_z>baty)[0]
        #BATY = np.array(new_z[ibat][0]*(-1))
        BATY = baty*(-1)
        print(BATY,baty)
        np.array(BATY).astype('>f4').tofile(Global_RUN+"run"+ifolder+"/bathy_1x1_1105m_testpool")
    else:
        print("run"+ifolder)

 24%|████████████▎                                       | 59/250 [00:00<00:00, 293.27it/s]

-14.63 14.63
-15.23 15.23
-15.83 15.83
-15.83 15.83
-14.64 14.64
-11.37 11.37
-14.34 14.34
-12.86 12.86
-16.42 16.42
-15.53 15.53
-16.12 16.12
-17.31 17.31
-23.55 23.55
-15.53 15.53
-11.96 11.96
-17.6 17.6
-7.52 7.52
-21.46 21.46
-16.11 16.11
-18.78 18.78
-14.04 14.04
-20.27 20.27
-19.09 19.09
-22.07 22.07
-20.88 20.88
-17.9 17.9
-14.34 14.34
-20.88 20.88
-21.77 21.77
-20.88 20.88
-22.06 22.06
-22.06 22.06
-17.6 17.6
-24.45 24.45
-11.37 11.37
-20.58 20.58
-21.47 21.47
-20.88 20.88
-8.41 8.41
-25.93 25.93
-21.18 21.18
-21.47 21.47
-25.63 25.63
-26.52 26.52
-19.98 19.98
-17.01 17.01
-18.19 18.19
-24.74 24.74
-24.44 24.44
-26.82 26.82
-21.47 21.47
-19.69 19.69
-17.6 17.6
-17.9 17.9
-19.69 19.69
-13.15 13.15
-18.79 18.79
-18.19 18.19
-22.35 22.35
-19.08 19.08


 50%|█████████████████████████▋                         | 126/250 [00:00<00:00, 316.23it/s]

-18.48 18.48
-21.47 21.47
-14.64 14.64
-26.52 26.52
-21.17 21.17
-20.58 20.58
-20.57 20.57
-20.27 20.27
-12.56 12.56
-23.54 23.54
-28.89 28.89
-36.62 36.62
-58.65 58.65
run_074
-67.0 67.0
-64.02 64.02
run_077
-56.59 56.59
run_079
run_080
-51.21 51.21
-64.62 64.62
-68.2 68.2
run_084
-67.6 67.6
run_086
run_087
run_088
-14.63 14.63
run_090
-25.92 25.92
-14.04 14.04
-61.65 61.65
-45.86 45.86
-67.6 67.6
run_096
run_097
run_098
run_099
run_100
run_101
run_102
run_103
run_104
-69.09 69.09
run_106
-63.12 63.12
-48.82 48.82
-56.86 56.86
-48.82 48.82
-52.39 52.39
-55.07 55.07
-65.2 65.2
-66.09 66.09
-65.2 65.2
-47.63 47.63
-57.16 57.16
-51.8 51.8
-46.14 46.14
-38.71 38.71
-58.06 58.06
-49.12 49.12
-54.19 54.19
-65.21 65.21
-44.96 44.96
-43.17 43.17
-39.91 39.91


 76%|██████████████████████████████████████▉            | 191/250 [00:00<00:00, 319.10it/s]

-42.29 42.29
-30.99 30.99
-41.99 41.99
-45.86 45.86
-49.14 49.14
-44.67 44.67
-36.34 36.34
-41.1 41.1
-44.38 44.38
-46.76 46.76
-55.7 55.7
-34.56 34.56
-31.28 31.28
-46.15 46.15
-49.72 49.72
-53.88 53.88
-58.34 58.34
-51.49 51.49
-51.49 51.49
-37.21 37.21
-46.74 46.74
-43.17 43.17
run_150
run_151
run_152
-44.04 44.04
run_154
run_155
-61.89 61.89
run_157
run_158
run_159
run_160
run_161
-61.91 61.91
-69.36 69.36
-53.27 53.27
-62.21 62.21
-59.23 59.23
-44.64 44.64
-41.06 41.06
-51.78 51.78
-40.47 40.47
-38.69 38.69
-50.59 50.59
-42.26 42.26
-36.61 36.61
-38.98 38.98
-51.48 51.48
-50.58 50.58
-57.14 57.14
-52.07 52.07
-52.67 52.67
-59.81 59.81
-29.46 29.46
-28.28 28.28
-30.66 30.66
-41.96 41.96
-43.15 43.15
-33.93 33.93
run_188
-37.5 37.5
-49.7 49.7
-69.05 69.05
-40.47 40.47


 89%|█████████████████████████████████████████████▍     | 223/250 [00:00<00:00, 289.91it/s]

-57.74 57.74
-62.2 62.2
-53.86 53.86
-34.83 34.83
-28.58 28.58
-55.36 55.36
-13.74 13.74
-56.54 56.54
run_201
-67.56 67.56
-69.06 69.06
-36.01 36.01
-44.94 44.94
-61.62 61.62
-62.81 62.81
-48.21 48.21
run_209
-41.96 41.96
run_211
run_212
-65.19 65.19
run_214
run_215
-65.47 65.47
run_217
run_218
-51.2 51.2
run_220
-61.33 61.33
run_222
run_223
run_224
run_225
-56.84 56.84
-67.87 67.87
-59.54 59.54
run_229
-55.97 55.97
-39.59 39.59
-65.79 65.79
-52.98 52.98
-44.34 44.34
-39.89 39.89
run_236
-53.0 53.0
-58.65 58.65
-48.53 48.53


100%|███████████████████████████████████████████████████| 250/250 [00:00<00:00, 289.83it/s]

-39.3 39.3
-67.0 67.0
-45.55 45.55
run_243
-41.39 41.39
-39.6 39.6
-41.38 41.38
-42.27 42.27
run_248
-60.71 60.71
-56.85 56.85


In [16]:
new_z

<xarray.DataArray 'depth' (depth: 70)>
array([ 0.5,  1.5,  2.5,  3.5,  4.5,  5.5,  6.5,  7.5,  8.5,  9.5, 10.5,
       11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5, 20.5, 21.5,
       22.5, 23.5, 24.5, 25.5, 26.5, 27.5, 28.5, 29.5, 30.5, 31.5, 32.5,
       33.5, 34.5, 35.5, 36.5, 37.5, 38.5, 39.5, 40.5, 41.5, 42.5, 43.5,
       44.5, 45.5, 46.5, 47.5, 48.5, 49.5, 50.5, 51.5, 52.5, 53.5, 54.5,
       55.5, 56.5, 57.5, 58.5, 59.5, 60.5, 61.5, 62.5, 63.5, 64.5, 65.5,
       66.5, 67.5, 68.5, 69.5])
Dimensions without coordinates: depth

In [19]:
stop here!

SyntaxError: invalid syntax (2745754519.py, line 1)

# Location CTD profiles

In [ ]:
plt.scatter(lon,lat,s=50,c=Time,cmap="jet")

In [ ]:
ifolder = '001'
acb_old = "/Users/houndegno/Documents/MATLAB/Data/SASSIE_DATA/MITgcm_experiments/1D_OJH/1D_ocean_ice_column/Global_RUN/run_"+str(ifolder)+"/mnc_test_0002/sice.0000000000.t001.nc"#state.0000000000.t001.nc"
#acb = "/Users/houndegno/Documents/MATLAB/Data/SASSIE_DATA/MITgcm_experiments/1D_OJH/1D_ocean_ice_column/Global_RUN/run_200/mnc_test_0001/tave.0000000000.t001.nc"

#tave.0000000000.t001.nc
acb = "/Users/houndegno/Documents/MATLAB/Data/SASSIE_DATA/MITgcm_experiments/1D_OJH/1D_ocean_ice_column/Global_RUN/run_"+str(ifolder)+"/mnc_test_0001/sice.0000000000.t001.nc"#state.0000000000.t001.nc"


In [ ]:
ds1 = xr.open_dataset(acb_old)
ds1

In [ ]:
ds2 = xr.open_dataset(acb)
ds2

In [ ]:
ds1.si_AREA.squeeze().plot(color='r')
ds2.si_AREA.squeeze().plot(color='b')

In [ ]:
stop here!

In [ ]:
np.nansum((ds2.si_AREA).squeeze())

In [ ]:
np.nansum((ds.si_AREA).squeeze())

In [ ]:
S = np.squeeze(ds.S)
T = np.squeeze(ds.Temp)
z = ds.Z
t = ds.T

In [ ]:
T.shape

In [ ]:
fig = plt.figure(figsize=[10,6], num=1);plt.clf();
a = plt.subplot(121)
plt.plot(temp[:,ifolder-1],pres[:,ifolder-1])
a.invert_yaxis()
b = plt.subplot(122)
plt.plot(SA[:,ifolder-1],pres[:,ifolder-1])
b.invert_yaxis()
fig = plt.figure(figsize=[25,6], num=2);plt.clf();
S.T.plot(cmap='cmo.haline')
#plt.plot(t,-MLD,'-w',label='MLD',linewidth=2.5)
fig = plt.figure(figsize=[25,6], num=3);plt.clf();
T.T.plot(cmap='cmo.thermal')
#plt.plot(t,-MLD,'-w',label='MLD',linewidth=2.5)

In [ ]:
fig = plt.figure(figsize=[25,10], num=1);plt.clf();
a = plt.subplot(111)
#plt.plot(temp[:,ifolder-1],pres[:,ifolder-1])
temp.plot(vmin=1.5,vmax=2.3)
a.invert_yaxis()

In [ ]:
temp

In [ ]:
f,ax = plt.subplots(1,1, figsize=[12,5]);
h=ax.pcolormesh(temp.time[230:250], -temp.depth, temp[:,230:250],\
             vmin=-1,vmax=0, cmap='jet')
ax.set_aspect('auto')
f.colorbar(h, ax=ax)

In [ ]:
pres.shape